## Construção de Pipelines e Tuning de Hiperparâmetros no Apache Spark

### Introdução
Neste notebook, exploraremos o uso de Pipelines para organizar o processo de modelagem e a técnica de Tuning de Hiperparâmetros para otimizar o desempenho de um modelo de rede neural utilizando o Multi-Layer Perceptron (MLP) no conjunto de dados iris.

Os Pipelines do Apache Spark permitem automatizar o fluxo de trabalho de Machine Learning, encadeando diversas etapas, como transformação de dados, treinamento de modelos e avaliação. O Tuning de Hiperparâmetros, por sua vez, envolve a seleção dos melhores parâmetros para um modelo, o que pode melhorar significativamente a sua performance.

### Desenvolvimento
#### Carregamento e Divisão dos Dados
Primeiro, carregamos o conjunto de dados iris.csv e o dividimos em conjuntos de treinamento e teste. A divisão é feita de forma que 70% dos dados sejam utilizados para treinar o modelo e 30% para avaliar o desempenho.

In [0]:
iris = spark.read.csv("/FileStore/tables/iris.csv",inferSchema=True, header=True)
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

#### Criação dos Estágios do Pipeline
Criamos os estágios que comporão o pipeline. Isso inclui a vetorização das características, a indexação das classes e a configuração do classificador MLP.

- Vetorização: Usamos o VectorAssembler para combinar as colunas de características (sepallength, sepalwidth, petallength, petalwidth) em um único vetor chamado independente.

- Indexação: O StringIndexer converte as classes de flores em valores numéricos, armazenados na coluna label.

- Classificador MLP: Configuramos o classificador de rede neural com MLP, especificando a arquitetura e o número máximo de iterações.

In [0]:
from pyspark.ml.feature import VectorAssembler
vector = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

Out[4]: VectorAssembler_2f011a92b368

In [0]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="label")

In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3], featuresCol="independente", labelCol="label")

#### Construção do Pipeline
Com os estágios definidos, construímos o pipeline que irá automatizar o processo de transformação de dados e treinamento do modelo.

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vector, indexer, mlp])

#### Configuração do Tuning de Hiperparâmetros com Cross-Validation
Para otimizar o modelo, configuramos um grid de hiperparâmetros, variando o número de iterações máximas (maxIter) e a arquitetura da rede (layers). Utilizamos o CrossValidator para testar essas combinações e selecionar a que melhor se ajusta aos dados de treino.

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000]).addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]]).build()
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=grid,evaluator=MulticlassClassificationEvaluator(),numFolds=3)  

#### Treinamento do Modelo com Cross-Validation
Treinamos o modelo utilizando o pipeline configurado e o cross-validation para ajustar os hiperparâmetros.

In [0]:
modelo = crossval.fit(irisTreino)

#### Avaliação do Modelo
Após o treinamento, fazemos previsões no conjunto de teste e avaliamos a performance do modelo utilizando a métrica de acurácia.

In [0]:
previsao = modelo.transform(irisTeste)
previsao.show()

+-----------+----------+-----------+----------+---------------+-----------------+-----+--------------------+--------------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|          class|     independente|label|       rawPrediction|         probability|prediction|
+-----------+----------+-----------+----------+---------------+-----------------+-----+--------------------+--------------------+----------+
|        4.4|       2.9|        1.4|       0.2|    Iris-setosa|[4.4,2.9,1.4,0.2]|  1.0|[-270.68116640044...|[3.32074702659803...|       1.0|
|        4.5|       2.3|        1.3|       0.3|    Iris-setosa|[4.5,2.3,1.3,0.3]|  1.0|[-270.68116640044...|[3.32074702662823...|       1.0|
|        4.6|       3.1|        1.5|       0.2|    Iris-setosa|[4.6,3.1,1.5,0.2]|  1.0|[-270.68116640044...|[3.32074702659803...|       1.0|
|        4.6|       3.2|        1.4|       0.2|    Iris-setosa|[4.6,3.2,1.4,0.2]|  1.0|[-270.68116640044...|[3.32074702659803...|       1.0|
|        4.7|

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9761904761904762


- Modelo eficaz: Uma acurácia de 97.62% sugere que o modelo é eficaz em classificar as espécies de flores baseadas nas características fornecidas (comprimento e largura das pétalas e sépalas).
- Boas escolhas de hiperparâmetros: O uso de Cross-Validation e o tuning de hiperparâmetros foram bem-sucedidos em encontrar uma configuração que maximiza a acurácia do modelo.
- Generalização: A alta acurácia também sugere que o modelo não está superajustado (overfitting) aos dados de treinamento, pois ele consegue performar bem em novos dados (conjunto de teste).

### Conclusão
Neste notebook, utilizamos o Apache Spark para construir um pipeline de Machine Learning que automatiza o processo de pré-processamento e modelagem, seguido por um tuning de hiperparâmetros utilizando cross-validation. Através dessa abordagem, conseguimos otimizar a performance do modelo de rede neural MLP, melhorando a acurácia na tarefa de classificação.

O uso de pipelines e tuning de hiperparâmetros é essencial para que o modelo de Machine Learning seja eficiente e generalize bem em novos dados.